In [1]:
import mne
import glob
import os
import scipy.io
from scipy.spatial.distance import cosine
import numpy as np
from pymatreader import read_mat
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from scipy.stats import wilcoxon


In [2]:
################################ HELPER FUNCTIONS ################################ 

# create condlist
conditionlist = {
    'leading_Barn': 1,
    'leading_beach': 2,
    'leading_library': 3,
    'leading_restaurant': 4, 
    'leading_cave': 5,
    'trailing_church': 6,
    'trailing_conference_room': 7,
    'trailing_castle': 8, 
    'trailing_forest': 9      
}


# create category dict
#Idea: we create a customised event_id dict that allows us to later just extract the trailing images corresponding to our transitional probabiliy categories

# Mapping VALID CONDITIONS (75%): 
    # leading_barn -> trailing_church
    # leading_beach -> trailing_church
    # leading_library -> trailing_conference_room
    # leading_restaurant ->  trailing_conference_room

# Mapping INVALID CONDITIONS (25%): 
    # leading_barn -> trailing_conference_room
    # leading_beach -> trailing_conference_room
    # leading_library -> trailing_church
    # leading_restaurant -> trailing_church
    
# Mapping CONTROL CONDITIONS (50%): 
    # leading_cave -> trailing_castle
    # leading_cave -> trailing_forest
    
category_dict = {
    0: (1, 6), # valid conditions 75 %
    1: (2, 6), # leading_beach -> trailing_church
    2: (3, 7), # leading_library -> trailing_conference_room
    3: (4, 7),  # leading_restaurant ->  trailing_conference_room
    4: (1, 7), # invalid conditions 0.25 %
    5: (2, 7),
    6: (3, 6), 
    7: (4, 6), 
    8: (5, 8), # control conditions 50 %
    9: (5, 9)
}


rdm_dict = {
    0: (0,1), # 0.75%
    1: (2,3), # 0. 75 % 
    2: (4,5), # 0.25 % 
    3: (6,7), # 0.25 %
    4: (8,), # 0.5 % 
    5: (9,)  # 0.5 %  
}






def convert_trialinfo(data, mapping):
    '''This function converts condition label strings into integer based on the mapping in conditionlist.
        Input: data, condition mapping
        Returns: List of integers
    '''    
    if "trialinfo" in data:
        trialinfo_labels = data["trialinfo"]
    else:
        raise KeyError("'trialinfo' field is missing in data['fD']")

    return np.array([mapping[cond] for cond in trialinfo_labels])


def map_events(event_id, category_dict):
    """
    This function creates a customised event_id dict that allows us to later just extract the trailing images 
    corresponding to our transitional probabiliy categories
    Maps events based on specified (leading, trailing) pairs in category_dict.
    
    Parameters:
    - event_id (list or array-like): List of event IDs.
    - category_dict (dict): Dictionary mapping keys to (leading, trailing) event pairs.
    
    Returns:
    - list: Mapped event categories based on leading-trailing pairs.
    """
    # Initialize event_maps with -1 for each event
    event_maps = [-1] * len(event_id)
    
    # Iterate over event IDs, starting from the second event
    for idx in range(1, len(event_id)):
        # Check if the current and previous event form a valid (leading, trailing) pair
        for key, (leading, trailing) in category_dict.items():
            if idx > 0 and (event_id[idx - 1] == leading) and (event_id[idx] == trailing):
                event_maps[idx] = key
    
    return event_maps




def get_unique_event_ids(event_id):
    """
    Retrieves unique event IDs, excluding the specified unwanted value (-1).
    
    Parameters:
    - event_id (list or array-like): List of event IDs.
    
    Returns:
    - numpy array: Array of unique event IDs, excluding -1.
    """
    # Get unique event IDs and filter out -1
    unique_event_ids = np.unique(event_id)
    unique_event_ids = unique_event_ids[unique_event_ids != -1]
    
    return unique_event_ids



# Define resultspath
results_path_eeg = "/Users/denisekittelmann/Documents/Python/BiMoL/results/rdms/"
#results_path_ann = 





In [3]:
################################ CREATE CUSTOMISED EVENT ID DICT ################################ 

#Idea: we create a customised event_id dict that allows us to later just extract the trailing images corresponding to our transitional probabiliy categories

# Mapping VALID CONDITIONS (75%): 
    # leading_barn -> trailing_church
    # leading_beach -> trailing_church
    # leading_library -> trailing_conference_room
    # leading_restaurant ->  trailing_conference_room

# Mapping INVALID CONDITIONS (25%): 
    # leading_barn -> trailing_conference_room
    # leading_beach -> trailing_conference_room
    # leading_library -> trailing_church
    # leading_restaurant -> trailing_church
    
# Mapping CONTROL CONDITIONS (50%): 
    # leading_cave -> trailing_castle
    # leading_cave -> trailing_forest
    
category_dict = {
    0: (1, 6), # valid conditions 75 %
    1: (2, 6), # leading_beach -> trailing_church
    2: (3, 7), # leading_library -> trailing_conference_room
    3: (4, 7),  # leading_restaurant ->  trailing_conference_room
    4: (1, 7), # invalid conditions 0.25 %
    5: (2, 7),
    6: (3, 6), 
    7: (4, 6), 
    8: (5, 8), # control conditions 50 %
    9: (5, 9)
}


rdm_dict = {
    0: (0,1), # 0.75%
    1: (2,3), # 0. 75 % 
    2: (4,5), # 0.25 % 
    3: (6,7), # 0.25 %
    4: (8,), # 0.5 % 
    5: (9,)  # 0.5 %  
}



                       
#event_id = event_maps



In [4]:


# Initialize a dictionary to store mean activations for each condition
# Assuming rdm_dict defines the conditions

dir = "/Users/denisekittelmann/Documents/Python/BiMoL/data/EEG/"

# Create iterator
ids = [pid for pid in range(1, 32) if pid != 20] 


mean_activations_early = {category: [] for category in rdm_dict.keys()}
mean_activations_late = {category: [] for category in rdm_dict.keys()}
mean_activations_b1 = {category: [] for category in rdm_dict.keys()}
mean_activations_b2 = {category: [] for category in rdm_dict.keys()}


# Loop over all participants
for pid in ids:
    print(pid)

    fname = os.path.join(dir, f"eTadff_sub{pid:02d}.mat")
    print(f"Loading data for participant {pid} from file {fname}")
    data = read_mat(fname)["fD"]

    data["trialinfo"] = convert_trialinfo(data, conditionlist)
    
    # read fields needed for MNE epochs
    epochs_data = np.array(data["trial"]) * 1e-6  # in microvolts
    tmin = data['time'][0][0]
    sfreq = int(1. / (data['time'][0][1] - tmin))
    ch_names = data["label"]
    ch_types = data["elec"]["chantype"]
    montage = mne.channels.make_standard_montage("biosemi64", head_size=0.095)

    event_id = list(data["trialinfo"])
    events = np.stack([np.arange(len(event_id)),
                       np.zeros_like(event_id), 
                       event_id], axis=1)

    # create the info field
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

    # Create an MNE Epochs object
    epochs = mne.EpochsArray(data=epochs_data, info=info, tmin=tmin,
                             events=events, event_id=event_id)
    epochs = epochs.set_montage(montage)
    
    event_id = map_events(event_id, category_dict)
    print("event_id:",event_id)
    
    early_start, early_end = 0.128, 0.180
    late_start, late_end = 0.280, 0.296
    
    epochs_early = epochs.copy().crop(tmin=early_start, tmax=early_end)
    epochs_late = epochs.copy().crop(tmin=late_start, tmax=late_end)
    b1 = epochs[:len(epochs) // 2]
    b2 = epochs[len(epochs) // 2:]
    

    # Loop over each condition and compute the mean activations
    for condition, idx_list in rdm_dict.items():
        if isinstance(idx_list, int):
            idx_list = [idx_list]

        # Early window
        epochs_condition_early = [epochs_early[event_name] for event_name in idx_list]
        mean_representation_early = np.mean([epoch.average().get_data().reshape(-1) for epoch in epochs_condition_early], axis=0)
        mean_activations_early[condition].append(mean_representation_early)

        # Late window
        epochs_condition_late = [epochs_late[event_name] for event_name in idx_list]
        mean_representation_late = np.mean([epoch.average().get_data().reshape(-1) for epoch in epochs_condition_late], axis=0)
        mean_activations_late[condition].append(mean_representation_late)

        # Block 1
        epochs_condition_b1 = [b1[event_name] for event_name in idx_list]
        mean_representation_b1 = np.mean([epoch.average().get_data().reshape(-1) for epoch in epochs_condition_b1], axis=0)
        mean_activations_b1[condition].append(mean_representation_b1) # für jede Vp mean in 

        # Block 2
        epochs_condition_b2 = [b2[event_name] for event_name in idx_list]
        mean_representation_b2 = np.mean([epoch.average().get_data().reshape(-1) for epoch in epochs_condition_b2], axis=0)
        mean_activations_b2[condition].append(mean_representation_b2)

        # Compute the grand average for each condition, time window, and block across participants
        final_mean_activations_early = {condition: np.mean(mean_activations_early[condition], axis=0) for condition in mean_activations_early}
        final_mean_activations_late = {condition: np.mean(mean_activations_late[condition], axis=0) for condition in mean_activations_late}
        final_mean_activations_b1 = {condition: np.mean(mean_activations_b1[condition], axis=0) for condition in mean_activations_b1}
        final_mean_activations_b2 = {condition: np.mean(mean_activations_b2[condition], axis=0) for condition in mean_activations_b2}

1
Loading data for participant 1 from file /Users/denisekittelmann/Documents/Python/BiMoL/data/EEG/eTadff_sub01.mat
Not setting metadata
3456 matching events found
No baseline correction applied
0 projection items activated
event_id: [-1, 2, -1, 3, -1, 0, -1, 1, -1, 9, -1, 9, -1, 9, -1, 8, -1, 1, -1, 4, -1, 1, -1, 2, -1, 1, -1, 9, -1, 1, -1, 2, -1, 2, -1, 2, -1, 3, -1, 3, -1, 1, -1, 8, -1, 8, -1, 9, -1, 7, -1, 2, -1, 7, -1, 3, -1, 8, -1, 9, -1, 0, -1, 9, -1, 3, -1, 8, -1, 9, -1, 8, -1, 1, -1, 7, -1, 8, -1, 9, -1, 8, -1, 3, -1, 8, -1, 8, -1, 9, -1, 1, -1, 1, -1, 1, -1, 0, -1, 0, -1, 3, -1, 5, -1, 2, -1, 8, -1, 8, -1, 8, -1, 8, -1, 8, -1, 1, -1, 2, -1, 1, -1, 8, -1, 1, -1, 0, -1, 4, -1, 1, -1, 3, -1, 1, -1, 9, -1, 3, -1, 1, -1, 6, -1, 2, -1, 8, -1, 1, -1, 8, -1, 5, -1, 1, -1, 8, -1, 2, -1, 9, -1, 8, -1, 9, -1, 6, -1, 1, -1, 6, -1, 9, -1, 3, -1, 5, -1, 0, -1, 3, -1, 6, -1, 3, -1, 1, -1, 9, -1, 6, -1, 8, -1, 8, -1, 9, -1, 8, -1, 1, -1, 0, -1, 9, -1, 4, -1, 2, -1, 0, -1, 6, -1, 1, -1, 6, -1

/Users/denisekittelmann/miniconda3/envs/mne/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/denisekittelmann/miniconda3/envs/mne/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2
Loading data for participant 2 from file /Users/denisekittelmann/Documents/Python/BiMoL/data/EEG/eTadff_sub02.mat
Not setting metadata
3456 matching events found
No baseline correction applied
0 projection items activated
event_id: [-1, 2, -1, 3, -1, 0, -1, 1, -1, 9, -1, 9, -1, 9, -1, 8, -1, 1, -1, 4, -1, 1, -1, 2, -1, 1, -1, 9, -1, 1, -1, 2, -1, 2, -1, 2, -1, 3, -1, 3, -1, 1, -1, 8, -1, 8, -1, 9, -1, 7, -1, 2, -1, 7, -1, 3, -1, 8, -1, 9, -1, 0, -1, 9, -1, 3, -1, 8, -1, 9, -1, 8, -1, 1, -1, 7, -1, 8, -1, 9, -1, 8, -1, 3, -1, 8, -1, 8, -1, 9, -1, 1, -1, 1, -1, 1, -1, 0, -1, 0, -1, 3, -1, 5, -1, 2, -1, 8, -1, 8, -1, 8, -1, 8, -1, 8, -1, 1, -1, 2, -1, 1, -1, 8, -1, 1, -1, 0, -1, 4, -1, 1, -1, 3, -1, 1, -1, 9, -1, 3, -1, 1, -1, 6, -1, 2, -1, 8, -1, 1, -1, 8, -1, 5, -1, 1, -1, 8, -1, 2, -1, 9, -1, 8, -1, 9, -1, 6, -1, 1, -1, 6, -1, 9, -1, 3, -1, 5, -1, 0, -1, 3, -1, 6, -1, 3, -1, 1, -1, 9, -1, 6, -1, 8, -1, 8, -1, 9, -1, 8, -1, 1, -1, 0, -1, 9, -1, 4, -1, 2, -1, 0, -1, 6, -1, 1, -1, 6, -1

In [17]:
len(mean_activations_b1[0][0])

11520

In [19]:
len(final_mean_activations_early[1])

1088

In [13]:
eeg_mean_early = {condition: np.mean(activation) for condition, activation in final_mean_activations_early.items()}
eeg_mean_early

{0: -5.1833147993732875e-08,
 1: -3.859437552333556e-08,
 2: -1.1653557746856929e-08,
 3: -1.6906348553663702e-08,
 4: -5.586891924927226e-08,
 5: -4.969060406283991e-08}

In [5]:
eeg_mean_late = {condition: np.mean(activation) for condition, activation in final_mean_activations_late.items()}
eeg_mean_early = {condition: np.mean(activation) for condition, activation in final_mean_activations_early.items()}

eeg_mean_b1 = {condition: np.mean(activation) for condition, activation in final_mean_activations_b1.items()}
eeg_mean_b2 = {condition: np.mean(activation) for condition, activation in final_mean_activations_b2.items()}


In [9]:
eeg_mean_late

eeg_mean_b1

eeg_mean_b2
eeg_mean_b1

{0: -4.5161124118655475e-08,
 1: -1.8330159319996984e-08,
 2: -2.4960150938052806e-08,
 3: -4.055338713346937e-09,
 4: -4.750306361140031e-08,
 5: -2.3860123907537655e-08}

In [10]:
# Save the dictionaries directly
savepath = "/Users/denisekittelmann/Documents/Python/BiMoL/results/mean_activations/"

np.save(f"{savepath}/eeg_mean_late.npy", eeg_mean_late)
np.save(f"{savepath}/eeg_mean_early.npy", eeg_mean_early)
np.save(f"{savepath}/eeg_mean_b1.npy", eeg_mean_b1)
np.save(f"{savepath}/eeg_mean_b2.npy", eeg_mean_b2)

print("Saved EEG mean activations as dictionaries.")


Saved EEG mean activations as dictionaries.


In [11]:
print(eeg_mean_late)
print(eeg_mean_early)
print(eeg_mean_b1)
print(eeg_mean_b2)

{0: -4.38522909897402e-08, 1: 1.8658510849922085e-08, 2: -1.088060202102489e-08, 3: 3.191398847584587e-08, 4: -6.46833455783602e-08, 5: 1.0867958650003197e-08}
{0: -5.1833147993732875e-08, 1: -3.859437552333556e-08, 2: -1.1653557746856929e-08, 3: -1.6906348553663702e-08, 4: -5.586891924927226e-08, 5: -4.969060406283991e-08}
{0: -4.5161124118655475e-08, 1: -1.8330159319996984e-08, 2: -2.4960150938052806e-08, 3: -4.055338713346937e-09, 4: -4.750306361140031e-08, 5: -2.3860123907537655e-08}
{0: -2.775482623880376e-08, 1: 5.768241852154429e-10, 2: -1.6597462880590604e-08, 3: -3.902580776894249e-08, 4: -6.975846740094997e-09, 5: -4.2385811532023415e-08}


In [21]:
print(eeg_mean_late)
print(eeg_mean_early)
print(eeg_mean_b1)
print(eeg_mean_b2)

{0: -4.38522909897402e-08, 1: 1.8658510849922085e-08, 2: -1.088060202102489e-08, 3: 3.191398847584587e-08, 4: -6.46833455783602e-08, 5: 1.0867958650003197e-08}
{0: -5.1833147993732875e-08, 1: -3.859437552333556e-08, 2: -1.1653557746856929e-08, 3: -1.6906348553663702e-08, 4: -5.586891924927226e-08, 5: -4.969060406283991e-08}
{0: -4.5161124118655475e-08, 1: -1.8330159319996984e-08, 2: -2.4960150938052806e-08, 3: -4.055338713346937e-09, 4: -4.750306361140031e-08, 5: -2.3860123907537655e-08}
{0: -2.775482623880376e-08, 1: 5.768241852154429e-10, 2: -1.6597462880590604e-08, 3: -3.902580776894249e-08, 4: -6.975846740094997e-09, 5: -4.2385811532023415e-08}


In [ ]:
# Load the dictionaries
eeg_mean_late = np.load(f"{savepath}/eeg_mean_late.npy", allow_pickle=True).item()
eeg_mean_early = np.load(f"{savepath}/eeg_mean_early.npy", allow_pickle=True).item()
eeg_mean_b1 = np.load(f"{savepath}/eeg_mean_b1.npy", allow_pickle=True).item()
eeg_mean_b2 = np.load(f"{savepath}/eeg_mean_b2.npy", allow_pickle=True).item()

print("Loaded EEG mean activation dictionaries:", eeg_mean_late)




In [2]:
final_bp_acts_mean_activations_final = np.load("/Users/denisekittelmann/Documents/Python/BiMoL/results/mean_activations/final_bp_acts_mean_activations_final.npy", allow_pickle=True)
final_pcn_acts_mean_activations_final = np.load("/Users/denisekittelmann/Documents/Python/BiMoL/results/mean_activations/final_pcn_acts_mean_activations_final.npy", allow_pickle=True)
final_pcn_error_mean_activations_final = np.load("/Users/denisekittelmann/Documents/Python/BiMoL/results/mean_activations/final_pcn_error_mean_activations_final.npy", allow_pickle=True)


print(final_bp_acts_mean_activations_final)
print(final_pcn_acts_mean_activations_final)
print(final_pcn_error_mean_activations_final)


{0: 0.49660692, 1: 0.49323025, 2: 0.50092524, 3: 0.4848873, 4: 0.48292464, 5: 0.48447466}
{0: -0.0043925494, 1: -2.9047185e-05, 2: 0.0023232477, 3: -0.006735514, 4: -0.0007191675, 5: -0.0061575836}
{0: 2.059048e-09, 1: 2.0195972e-09, 2: 2.2514746e-09, 3: 1.9690216e-09, 4: 2.2758129e-09, 5: 2.2847468e-09}


In [ ]:
final_pcn_acts_mean_activations_final_dict = final_pcn_acts_mean_activations_final.item()

array({0: -0.0043925494, 1: -2.9047185e-05, 2: 0.0023232477, 3: -0.006735514, 4: -0.0007191675, 5: -0.0061575836},
      dtype=object)

In [10]:
# Extract the dictionary
final_bp_acts_mean_activations_final_dict = final_bp_acts_mean_activations_final.item()
print(final_bp_acts_mean_activations_final_dict)

final_pcn_error_mean_activations_final_dict = final_pcn_error_mean_activations_final.item()
print(final_pcn_error_mean_activations_final_dict)


final_pcn_acts_mean_activations_final_dict = final_pcn_acts_mean_activations_final.item()
print(final_pcn_acts_mean_activations_final_dict)






{0: 0.49660692, 1: 0.49323025, 2: 0.50092524, 3: 0.4848873, 4: 0.48292464, 5: 0.48447466}
{0: 2.059048e-09, 1: 2.0195972e-09, 2: 2.2514746e-09, 3: 1.9690216e-09, 4: 2.2758129e-09, 5: 2.2847468e-09}
{0: -0.0043925494, 1: -2.9047185e-05, 2: 0.0023232477, 3: -0.006735514, 4: -0.0007191675, 5: -0.0061575836}


In [4]:
eeg_mean_late = np.load("/Users/denisekittelmann/Documents/Python/BiMoL/results/mean_activations/eeg_mean_late.npy", allow_pickle=True)
eeg_mean_early = np.load("/Users/denisekittelmann/Documents/Python/BiMoL/results/mean_activations/eeg_mean_early.npy", allow_pickle=True)
eeg_mean_b1 = np.load("/Users/denisekittelmann/Documents/Python/BiMoL/results/mean_activations/eeg_mean_b1.npy", allow_pickle=True)
eeg_mean_b2 = np.load("/Users/denisekittelmann/Documents/Python/BiMoL/results/mean_activations/eeg_mean_b2.npy", allow_pickle=True)

print(eeg_mean_late)
print(eeg_mean_early)
print(eeg_mean_b1)
print(eeg_mean_b2)

{0: -4.38522909897402e-08, 1: 1.8658510849922085e-08, 2: -1.088060202102489e-08, 3: 3.191398847584587e-08, 4: -6.46833455783602e-08, 5: 1.0867958650003197e-08}
{0: -5.1833147993732875e-08, 1: -3.859437552333556e-08, 2: -1.1653557746856929e-08, 3: -1.6906348553663702e-08, 4: -5.586891924927226e-08, 5: -4.969060406283991e-08}
{0: -4.5161124118655475e-08, 1: -1.8330159319996984e-08, 2: -2.4960150938052806e-08, 3: -4.055338713346937e-09, 4: -4.750306361140031e-08, 5: -2.3860123907537655e-08}
{0: -2.775482623880376e-08, 1: 5.768241852154429e-10, 2: -1.6597462880590604e-08, 3: -3.902580776894249e-08, 4: -6.975846740094997e-09, 5: -4.2385811532023415e-08}


In [6]:
eeg_mean_late = eeg_mean_late.item()
print(eeg_mean_late)

eeg_mean_early = eeg_mean_early.item()
print(eeg_mean_early)


eeg_mean_b1 = eeg_mean_b1.item()
print(eeg_mean_b1)


eeg_mean_b2 = eeg_mean_b2.item()
print(eeg_mean_b2)

{0: -4.38522909897402e-08, 1: 1.8658510849922085e-08, 2: -1.088060202102489e-08, 3: 3.191398847584587e-08, 4: -6.46833455783602e-08, 5: 1.0867958650003197e-08}
{0: -5.1833147993732875e-08, 1: -3.859437552333556e-08, 2: -1.1653557746856929e-08, 3: -1.6906348553663702e-08, 4: -5.586891924927226e-08, 5: -4.969060406283991e-08}
{0: -4.5161124118655475e-08, 1: -1.8330159319996984e-08, 2: -2.4960150938052806e-08, 3: -4.055338713346937e-09, 4: -4.750306361140031e-08, 5: -2.3860123907537655e-08}
{0: -2.775482623880376e-08, 1: 5.768241852154429e-10, 2: -1.6597462880590604e-08, 3: -3.902580776894249e-08, 4: -6.975846740094997e-09, 5: -4.2385811532023415e-08}


In [20]:
final_pcn_acts_mean_activations_final_dict = final_pcn_acts_mean_activations_final.item()

In [21]:
final_pcn_acts_mean_activations_final_dict

{0: -0.0043925494,
 1: -2.9047185e-05,
 2: 0.0023232477,
 3: -0.006735514,
 4: -0.0007191675,
 5: -0.0061575836}

In [36]:
common_categories = set(eeg_mean_late.keys()).intersection(final_pcn_acts_mean_activations_final_dict.keys())

# Extract aligned values
eeg_late_values = [eeg_mean_late[k] for k in common_categories]
network_values = [final_pcn_acts_mean_activations_final_dict[k] for k in common_categories]

print("Aligned EEG values:", eeg_late_values)
print("Aligned network values:", network_values)  



common_categories = set(eeg_mean_late.keys()).intersection(final_bp_acts_mean_activations_final_dict.keys())

# Extract aligned values
eeg_late_values = [eeg_mean_late[k] for k in common_categories]
network_values_bp = [final_bp_acts_mean_activations_final_dict[k] for k in common_categories]

print("Aligned EEG values:", eeg_late_values)
print("Aligned network values:", network_values_bp) 



# Extract aligned values
eeg_late_values = [eeg_mean_late[k] for k in common_categories]
network_values_bp = [final_bp_acts_mean_activations_final_dict[k] for k in common_categories]

print("Aligned EEG values:", eeg_late_values)
print("Aligned network values:", network_values_bp) 




# Extract aligned values
eeg_early_values = [eeg_mean_early[k] for k in common_categories]
network_values_pcn = [final_pcn_acts_mean_activations_final_dict[k] for k in common_categories]

print("Aligned EEG values:", eeg_late_values)
print("Aligned network values:", network_values_pcn) 



# Extract aligned values
eeg_early_values = [eeg_mean_early[k] for k in common_categories]
network_values_pcn = [final_pcn_acts_mean_activations_final_dict[k] for k in common_categories]

print("Aligned EEG values:", eeg_late_values)
print("Aligned network values:", network_values_pcn) 



eeg_b1_values = [eeg_mean_b1[k] for k in common_categories]
network_values_pcn = [final_pcn_acts_mean_activations_final_dict[k] for k in common_categories]

print("Aligned EEG values:", eeg_b1_values)
print("Aligned network values:", eeg_b1_values) 




eeg_b2_values = [eeg_mean_b2[k] for k in common_categories]
network_values_pcn = [final_pcn_acts_mean_activations_final_dict[k] for k in common_categories]

print("Aligned EEG values:", eeg_b2_values)
print("Aligned network values:", network_values_pcn) 

Aligned EEG values: [-4.38522909897402e-08, 1.8658510849922085e-08, -1.088060202102489e-08, 3.191398847584587e-08, -6.46833455783602e-08, 1.0867958650003197e-08]
Aligned network values: [-0.0043925494, -2.9047185e-05, 0.0023232477, -0.006735514, -0.0007191675, -0.0061575836]
Aligned EEG values: [-4.38522909897402e-08, 1.8658510849922085e-08, -1.088060202102489e-08, 3.191398847584587e-08, -6.46833455783602e-08, 1.0867958650003197e-08]
Aligned network values: [0.49660692, 0.49323025, 0.50092524, 0.4848873, 0.48292464, 0.48447466]
Aligned EEG values: [-4.38522909897402e-08, 1.8658510849922085e-08, -1.088060202102489e-08, 3.191398847584587e-08, -6.46833455783602e-08, 1.0867958650003197e-08]
Aligned network values: [0.49660692, 0.49323025, 0.50092524, 0.4848873, 0.48292464, 0.48447466]
Aligned EEG values: [-4.38522909897402e-08, 1.8658510849922085e-08, -1.088060202102489e-08, 3.191398847584587e-08, -6.46833455783602e-08, 1.0867958650003197e-08]
Aligned network values: [-0.0043925494, -2.904

In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation
spearman_corr, spearman_p = spearmanr(eeg_late_values, network_values)

# Print the results
print(f"Spearman correlation late PCN: {spearman_corr}, p-value: {spearman_p}")


spearman_corr, spearman_p = spearmanr(eeg_late_values, network_values_bp)

# Print the results
print(f"Spearman correlation late BP: {spearman_corr}, p-value: {spearman_p}") 


spearman_corr, spearman_p = spearmanr(eeg_early_values, network_values_pcn)

# Print the results
print(f"Spearman correlation early PCN: {spearman_corr}, p-value: {spearman_p}")


spearman_corr, spearman_p = spearmanr(eeg_early_values, network_values_bp)

# Print the results
print(f"Spearman correlation early BP: {spearman_corr}, p-value: {spearman_p}")



spearman_corr, spearman_p = spearmanr(eeg_b1_values, network_values_pcn)

# Print the results
print(f"Spearman correlation b1 PCN: {spearman_corr}, p-value: {spearman_p}")


spearman_corr, spearman_p = spearmanr(eeg_b1_values, network_values_bp)

# Print the results
print(f"Spearman correlation b1 BP: {spearman_corr}, p-value: {spearman_p}")



print(f"Spearman correlation b1 BP: {spearman_corr}, p-value: {spearman_p}")



spearman_corr, spearman_p = spearmanr(eeg_b2_values, network_values_pcn)

# Print the results
print(f"Spearman correlation b2 pcn: {spearman_corr}, p-value: {spearman_p}")


spearman_corr, spearman_p = spearmanr(eeg_b2_values, network_values_bp)

# Print the results
print(f"Spearman correlation b2 bp: {spearman_corr}, p-value: {spearman_p}")


Spearman correlation late PCN: -0.3714285714285715, p-value: 0.46847813411078715
Spearman correlation late BP: 0.08571428571428573, p-value: 0.8717434402332361
Spearman correlation early PCN: 0.2, p-value: 0.704
Spearman correlation early BP: 0.6, p-value: 0.20799999999999982
Spearman correlation b1 pcn: -0.3714285714285715, p-value: 0.46847813411078715
Spearman correlation b1 bp: 0.08571428571428573, p-value: 0.8717434402332361
Spearman correlation early BP: 0.08571428571428573, p-value: 0.8717434402332361
Spearman correlation b2 pcn: 0.7714285714285715, p-value: 0.07239650145772594
Spearman correlation b2 bp: 0.14285714285714288, p-value: 0.7871720116618075
